In [49]:
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st
import pandas as pd
import plotly.io as pio

pio.templates.default = None

from mainnet_launch.constants import AutopoolConstants, AUTO_ETH

from mainnet_launch.database.schema.full import (
    Blocks,
    DestinationStates,
    Destinations,
    AutopoolDestinationStates,
    AutopoolDestinations,
)
from mainnet_launch.database.schema.postgres_operations import (
    merge_tables_as_df,
    TableSelector,
)

autopool = AUTO_ETH

destination_state_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            DestinationStates,
        ),
        TableSelector(
            Destinations,
            select_fields=[
                Destinations.underlying_symbol,
                Destinations.exchange_name,
            ],
            join_on=(Destinations.destination_vault_address == DestinationStates.destination_vault_address)
            & (Destinations.chain_id == DestinationStates.chain_id),
        ),
        TableSelector(
            AutopoolDestinations,
            select_fields=[
                AutopoolDestinations.autopool_vault_address,
            ],
            join_on=(AutopoolDestinations.destination_vault_address == DestinationStates.destination_vault_address)
            & (AutopoolDestinations.chain_id == DestinationStates.chain_id),
        ),
        TableSelector(
            Blocks,
            [Blocks.datetime],
            (DestinationStates.block == Blocks.block) & (DestinationStates.chain_id == Blocks.chain_id),
        ),
    ],
    where_clause=(AutopoolDestinations.autopool_vault_address == autopool.autopool_eth_addr),
    order_by=Blocks.datetime,
)


destination_state_df["readable_name"] = destination_state_df.apply(
    lambda row: f"{row['underlying_symbol']} ({row['exchange_name']})", axis=1
)

2025-05-22 17:10:16,036 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-22 17:10:16,037 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-22 17:10:16,037 INFO sqlalchemy.engine.Engine [cached since 728.5s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x13bbd3520>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-22 17:10:16,037 INFO sqlalchemy.engine.Engine SELECT
    destination_states.*,
    destinations.underlying_symbol,
    destinations.exchange_name,
    autopool_destinations.

In [50]:
destination_state_df["readable_name"].value_counts()

readable_name
osETH/wETH-BPT (balancer)       747
wstETH-WETH-BPT (balancer)      747
frxeth-ng-f (curve)             747
pxethweth (curve)               747
pxETH/wETH (balancer)           747
osETH-rETH (curve)              747
ECLP-wstETH-wETH (balancer)     747
B-rETH-STABLE (balancer)        747
ethx-f (curve)                  747
ETHx/wstETH (balancer)          747
wETHrETH (curve)                747
ECLP-wstETH-cbETH (balancer)    747
pxsteth (curve)                 747
OETHCRV-f (curve)               498
weETH/rETH (balancer)           249
OETH/WETH (curve)               249
weeth-ng (curve)                249
Name: count, dtype: int64

In [ ]:
destination_state_df = destination_state_df.groupby(["readable_name", "datetime"]).max().reset_index()

one_destination_df = 100 * destination_state_df[destination_state_df["readable_name"] == "ETHx/wstETH (balancer)"][
    ["incentive_apr", "fee_apr", "base_apr", "points_apr", "fee_plus_base_apr", "datetime"]
].set_index("datetime").astype(float)
px.line(one_destination_df)

,readable_name,datetime,destination_vault_address,block,chain_id,incentive_apr,fee_apr,base_apr,points_apr,fee_plus_base_apr,...,underlying_token_total_supply,safe_total_supply,lp_token_spot_price,lp_token_safe_price,from_rebalance_plan,rebalance_plan_timestamp,rebalance_plan_key,underlying_symbol,exchange_name,autopool_vault_address
0,B-rETH-STABLE (balancer),2024-09-15 23:59:59+00:00,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20759464,1,0.041235,0.003997,0.005817,0.0,NaN,...,13157.451200,12302.611124,1.036516,1.036523,False,NaN,NaN,B-rETH-STABLE,balancer,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
1,B-rETH-STABLE (balancer),2024-09-16 23:59:59+00:00,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20766617,1,0.040938,0.003618,0.005802,0.0,NaN,...,13157.496413,12383.288934,1.036121,1.036163,False,NaN,NaN,B-rETH-STABLE,balancer,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
2,B-rETH-STABLE (balancer),2024-09-17 23:59:59+00:00,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20773761,1,0.040507,0.003295,0.005848,0.0,NaN,...,13157.496413,12383.288934,1.036456,1.036462,False,NaN,NaN,B-rETH-STABLE,balancer,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
3,B-rETH-STABLE (balancer),2024-09-18 23:59:59+00:00,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20780916,1,0.040196,0.003022,0.005817,0.0,NaN,...,13268.458946,12479.834854,1.036477,1.036491,False,NaN,NaN,B-rETH-STABLE,balancer,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
4,B-rETH-STABLE (balancer),2024-09-19 23:59:59+00:00,0xf3ae3c74EaD129e770A864CeE291A805b170bBe0,20788084,1,0.040448,0.002794,0.005885,0.0,NaN,...,13268.526874,12494.174709,1.036502,1.036501,False,NaN,NaN,B-rETH-STABLE,balancer,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4228,wstETH-WETH-BPT (balancer),2025-05-17 23:59:59+00:00,0x84844d8A8a13337b31CF01DC384c74C862FCd903,22506180,1,NaN,NaN,NaN,0.0,NaN,...,9021.150222,NaN,1.044913,1.044913,False,NaN,NaN,wstETH-WETH-BPT,balancer,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
4229,wstETH-WETH-BPT (balancer),2025-05-18 23:59:59+00:00,0x84844d8A8a13337b31CF01DC384c74C862FCd903,22513304,1,NaN,NaN,NaN,0.0,NaN,...,9021.462804,NaN,1.044920,1.044930,False,NaN,NaN,wstETH-WETH-BPT,balancer,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
4230,wstETH-WETH-BPT (balancer),2025-05-19 23:59:59+00:00,0x84844d8A8a13337b31CF01DC384c74C862FCd903,22520426,1,NaN,NaN,NaN,0.0,NaN,...,9015.789211,NaN,1.044958,1.044799,False,NaN,NaN,wstETH-WETH-BPT,balancer,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
4231,wstETH-WETH-BPT (balancer),2025-05-20 23:59:59+00:00,0x84844d8A8a13337b31CF01DC384c74C862FCd903,22527557,1,NaN,NaN,NaN,0.0,NaN,...,9013.711978,NaN,1.045266,1.045578,False,NaN,NaN,wstETH-WETH-BPT,balancer,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56


In [ ]:
summary_stats_df = 100 * pd.pivot(destination_state_df, columns="readable_name", values="", index="datetime")
px.line(summary_stats_df)

In [54]:
summary_stats_df = 100 * pd.pivot(
    destination_state_df, columns="readable_name", values="incentive_apr", index="datetime"
)
px.line(summary_stats_df)